In [1]:
import re

def sanitize_markdown(md: str) -> str:
    # 1) Invisibles y normalización de saltos
    md = md.replace('\u00A0', ' ')   # NBSP → espacio
    md = md.replace('\u200B', '')    # ZWSP → nada
    md = md.replace('\r\n', '\n').replace('\r', '\n')

    # 2) Quitar sangría accidental de líneas no-code (pero conservar listas y tablas)
    def _desindent(line):
        if line.lstrip().startswith(('|','- ','* ','+ ', '1. ', '2. ', '3. ')):
            return line  # no tocar listas/tablas
        if line.startswith('    ') or line.startswith('\t'):
            # si parece bloque de código por sangría, pero no está dentro de ```...```, desindentamos 1 nivel
            return re.sub(r'^(    |\t)', '', line)
        return line
    lines = md.split('\n')
    # no tocamos dentro de fences ```
    out, in_code = [], False
    for ln in lines:
        if ln.strip().startswith('```'):
            in_code = not in_code
            out.append(ln)
            continue
        out.append(ln if in_code else _desindent(ln))
    md = '\n'.join(out)

    # 3) Escapar { } dentro de inline math $...$
    def esc_braces_in_math(m):
        body = m.group(1)
        body = body.replace('{', r'\{').replace('}', r'\}')
        return f'${body}$'
    md = re.sub(r'\$([^\$]+)\$', esc_braces_in_math, md)

    # 4) Asegurar líneas en blanco alrededor de $$...$$ y \[...\]
    def ensure_blank_around_block(math_block_pattern, text):
        # agrega línea en blanco antes y después si falta
        text = re.sub(rf'([^\n])\n({math_block_pattern})', r'\1\n\n\2', text)
        text = re.sub(rf'({math_block_pattern})\n([^\n])', r'\1\n\n\2', text)
        return text

    md = ensure_blank_around_block(r'\$\$[\s\S]*?\$\$', md)
    md = ensure_blank_around_block(r'\\\[[\s\S]*?\\\]', md)

    # 5) Asegurar línea en blanco antes y después de tablas (fila con pipes)
    md = re.sub(r'([^\n])\n(\|.*\|)', r'\1\n\n\2', md)
    md = re.sub(r'(\|.*\|)\n([^\n])', r'\1\n\n\2', md)

    # 6) Normalizar triples guiones de HR: que estén solos en la línea con blanco alrededor
    md = re.sub(r'([^\n])\n(---)\n([^\n])', r'\1\n\n---\n\n\3', md)

    return md

# Ejemplo de uso:
# texto = """<PEGA ACÁ TU CELDA QUE FALLA>"""
# print(sanitize_markdown(texto))


In [2]:
text ="""# Introducción a los Anillos

## Intuición
Un **anillo** es una estructura algebraica que generaliza las operaciones que conocemos de los números enteros: se puede **sumar**, **restar** y **multiplicar**, pero **no necesariamente dividir**.

## Definición formal
Un **anillo** es un conjunto \(R\) junto con dos operaciones (suma y producto) que satisfacen:

### Axiomas de la suma
1. \((R, +)\) es un **grupo abeliano**:
   - **Cerradura:** si \(a,b\in R\), entonces \(a+b\in R\).
   - **Asociatividad:** \((a+b)+c = a+(b+c)\).
   - **Elemento neutro aditivo:** existe \(0\in R\) tal que \(a+0=a\).
   - **Inverso aditivo:** para todo \(a\in R\), existe \(-a\) tal que \(a+(-a)=0\).
   - **Conmutatividad:** \(a+b=b+a\).

### Axiomas del producto
2. \((R,\cdot)\) es un **semigrupo**:
   - **Cerradura:** si \(a,b\in R\), entonces \(a\cdot b\in R\).
   - **Asociatividad:** \((a\cdot b)\cdot c = a\cdot (b\cdot c)\).

3. **Distributividad** de la multiplicación respecto a la suma:

\[
a(b+c)=ab+ac,\qquad (a+b)c=ac+bc
\]

## Tipos de anillos
| Tipo | Condición adicional |
|------|----------------------|
| **Anillo con unidad** | Existe \(1\neq 0\) tal que \(1\cdot a=a\cdot1=a\) |
| **Anillo conmutativo** | \(ab=ba\) para todo \(a,b\) |
| **Dominio de integridad** | Conmutativo, con \(1\neq 0\) y **sin divisores de cero** |
| **Cuerpo** | Todo elemento no nulo es **invertible** |
| **Dominio euclídeo** | Dominio de integridad con división con resto |

## Ejemplos clásicos
| Estructura | ¿Con unidad? | ¿Conmutativo? | ¿Divisores de cero? | Observaciones |
|-------------|:-----------:|:-------------:|:------------------:|---------------|
| \(\mathbb{Z}\) | ✅ | ✅ | ❌ | Anillo con unidad, dominio euclídeo |
| \(2\mathbb{Z}\) | ❌ | ✅ | ❌ | Anillo sin unidad |
| \(\mathbb{Z}_6\) | ✅ | ✅ | ✅ | Tiene divisores de cero (\(2\cdot3=0\)) |
| \(M_2(\mathbb{R})\) | ✅ | ❌ | ❌ | Anillo de matrices reales |
| \(\mathbb{Z}[i]\) | ✅ | ✅ | ❌ | Enteros gaussianos, dominio euclídeo |
| \(\mathbb{R}[x]\) | ✅ | ✅ | ❌ | Polinomios reales, dominio euclídeo |
| \(\mathbb{Z}_p\) con \(p\) primo | ✅ | ✅ | ❌ | Campo finito (todo no nulo tiene inverso) |

## Resumen visual
- Todos los **cuerpos** son **dominios de integridad**.
- Todos los **dominios de integridad** son **anillos conmutativos con unidad**.
- No todos los **anillos conmutativos** son **dominios** (p. ej. \(\mathbb{Z}_6\)).
- No todos los **anillos** tienen unidad (p. ej. \(2\mathbb{Z}\)).
"""
print(sanitize_markdown(text))

# Introducción a los Anillos

## Intuición
Un **anillo** es una estructura algebraica que generaliza las operaciones que conocemos de los números enteros: se puede **sumar**, **restar** y **multiplicar**, pero **no necesariamente dividir**.

## Definición formal
Un **anillo** es un conjunto \(R\) junto con dos operaciones (suma y producto) que satisfacen:

### Axiomas de la suma
1. \((R, +)\) es un **grupo abeliano**:
   - **Cerradura:** si \(a,b\in R\), entonces \(a+b\in R\).
   - **Asociatividad:** \((a+b)+c = a+(b+c)\).
   - **Elemento neutro aditivo:** existe \(0\in R\) tal que \(a+0=a\).
   - **Inverso aditivo:** para todo \(a\in R\), existe \(-a\) tal que \(a+(-a)=0\).
   - **Conmutatividad:** \(a+b=b+a\).

### Axiomas del producto
2. \((R,\cdot)\) es un **semigrupo**:
   - **Cerradura:** si \(a,b\in R\), entonces \(a\cdot b\in R\).
   - **Asociatividad:** \((a\cdot b)\cdot c = a\cdot (b\cdot c)\).

3. **Distributividad** de la multiplicación respecto a la suma:

\[
a(b+c)=ab+ac,

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
/tmp/ipykernel_3326/3697390695.py:1: SyntaxWarning: invalid escape sequence '\('
  text ="""# Introducción a los Anillos
